In [1]:
from getpass import getpass

openai_api_key = getpass("Enter OpenAI Api key: ")
serper_api_key = getpass('Enter Serper Api key:')

In [2]:
from haystack.dataclasses import Document

documents = [
    Document(
        content="""
        Munich, the vibrant capital of Bavaria in southern Germany, exudes a perfect blend of rich cultural
        heritage and modern urban sophistication. Nestled along the banks of the Isar River, Munich is renowned
        for its splendid architecture, including the iconic Neues Rathaus (New Town Hall) at Marienplatz and
        the grandeur of Nymphenburg Palace. The city is a haven for art enthusiasts, with world-class museums like the
        Alte Pinakothek housing masterpieces by renowned artists. Munich is also famous for its lively beer gardens, where
        locals and tourists gather to enjoy the city's famed beers and traditional Bavarian cuisine. The city's annual
        Oktoberfest celebration, the world's largest beer festival, attracts millions of visitors from around the globe.
        Beyond its cultural and culinary delights, Munich offers picturesque parks like the English Garden, providing a
        serene escape within the heart of the bustling metropolis. Visitors are charmed by Munich's warm hospitality,
        making it a must-visit destination for travelers seeking a taste of both old-world charm and contemporary allure.
        """
    )
]

In [3]:
from haystack.components.builders.prompt_builder import PromptBuilder
from haystack.components.generators.openai import OpenAIGenerator
from haystack.components.websearch.serper_dev import SerperDevWebSearch

prompt_template = """
Answer the following query given the douments.
If the answer is not contained within the documents reply with 'no_answer'
Query: {{ query }}
Documents:
{% for document in documents %}
  {{ document.content }}
{% endfor %}
"""

prompt_builder = PromptBuilder(template=prompt_template)
llm = OpenAIGenerator(api_key=openai_api_key)

websearch = SerperDevWebSearch(api_key=serper_api_key)

prompt_builder_from_search = PromptBuilder(template=prompt_template)
llm_from_search = OpenAIGenerator(api_key=openai_api_key)

In [4]:
from haystack.components.routers import ConditionalRouter

routes = [
    {
        "condition": "{{'no_answer' in replies[0]}}",
        "output": "{{query}}",
        "output_name": "go_to_websearch",
        "output_type": str,
    },
    {
        "condition": "{{'no_answer' not in replies[0]}}",
        "output": "{{replies[0]}}",
        "output_name": "answer",
        "output_type": str,
    },
]

router = ConditionalRouter(routes)

In [5]:
from haystack import Pipeline


pipe = Pipeline()
pipe.add_component("prompt_builder", prompt_builder)
pipe.add_component("llm", llm)
pipe.add_component("router", router)
pipe.add_component("websearch", websearch)
pipe.add_component("prompt_builder_from_search", prompt_builder_from_search)
pipe.add_component("llm_from_search", llm_from_search)

pipe.connect("prompt_builder", "llm")
pipe.connect("llm.replies", "router.replies")
pipe.connect("router.go_to_websearch", "websearch.query")
pipe.connect("router.go_to_websearch", "prompt_builder_from_search.query")
pipe.connect("websearch.documents", "prompt_builder_from_search.documents")
pipe.connect("prompt_builder_from_search", "llm_from_search")

In [6]:
pipe.draw('pipeline.png')

In [7]:
query = "Where is Munich?"

pipe.run({"prompt_builder":{"query": query, "documents": documents}, "router":{"query":query}})

{'llm': {'meta': [{'model': 'gpt-3.5-turbo-0613',
    'index': 0,
    'finish_reason': 'stop',
    'usage': {'completion_tokens': 16,
     'prompt_tokens': 271,
     'total_tokens': 287}}]},
 'router': {'answer': 'Munich is the capital of Bavaria and is located in southern Germany.'}}

In [9]:
query = "How many people live in Munich?"
pipe.run({"prompt_builder":{"query": query, "documents": documents}, "router":{"query":query}})

{'llm': {'meta': [{'model': 'gpt-3.5-turbo-0613',
    'index': 0,
    'finish_reason': 'stop',
    'usage': {'completion_tokens': 2,
     'prompt_tokens': 274,
     'total_tokens': 276}}]},
 'websearch': {'links': ['https://en.wikipedia.org/wiki/Munich',
   'https://worldpopulationreview.com/world-cities/munich-population',
   'https://en.wikipedia.org/wiki/Demographics_of_Munich',
   'https://www.macrotrends.net/cities/204371/munich/population',
   'https://www.statista.com/statistics/519723/munich-population-by-age-group/',
   'https://www.citypopulation.de/en/germany/bayern/m%C3%BCnchen_stadt/09162000__m%C3%BCnchen/',
   'https://www.britannica.com/place/Munich-Bavaria-Germany',
   'https://www.quora.com/How-many-people-live-in-Munich',
   'https://earth.esa.int/web/earth-watching/image-of-the-week/content/-/article/munich-germany/',
   'https://www.coe.int/en/web/interculturalcities/munich']},
 'llm_from_search': {'replies': ['The answer is 1,456,039.'],
  'meta': [{'model': 'gpt-3